# 路径

In [1]:
def setProjectPath(projectPath):
    import os
    import sys
    root=os.path.abspath(projectPath)
    print(root)
    sys.path.append(root)
    os.environ["TORCH_HOME"]=r"E:\Data\torch-model"
    import warnings  
    warnings.filterwarnings("ignore", category=RuntimeWarning, module="mne")

projectPath=r'D:\project_meta\NNproject\NNI'
setProjectPath(projectPath)

D:\project_meta\NNproject\NNI


In [2]:
import mne
datasetFolder=r'D:\dataset\driver_dataset\DROZY\DROZY'
outputFolder=r'D:\project_meta\NNproject\NNI\output'
from project.dataprocess.FolderTree import *
datasetDict=getDataPath(datasetFolder)
outputDict=getOutPath(outputFolder)
import pprint
pprint.pprint(datasetDict)
pprint.pprint(outputDict)

{'annotation_auto_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-auto',
 'annotation_manual_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\annotations-manual',
 'kinect_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\kinect-intrinsics.yaml',
 'kss_file': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\KSS.txt',
 'psg_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\psg',
 'pvt_rt_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\pvt-rt',
 'videos_folder': 'D:\\dataset\\driver_dataset\\DROZY\\DROZY\\videos_i8'}
{'ECG_filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\ECG',
 'ECG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\ECG_table.csv',
 'ECG_uniformfilted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\ECG',
 'EEG_filted_folder': 'D:\\project_meta\\NNproject\\NNI\\output\\psg_filted\\EEG',
 'EEG_table_file': 'D:\\project_meta\\NNproject\\NNI\\output\\EEG_table.csv',
 'EEG_uniformfilted_folder': 'D:\\project_

# 数据集

getBand

In [3]:
from project.dataprocess.DatasetProcess import getBandPower,getUniformBandPower,MakePeriodTable,MakeNonCrossTable,getBandPowerTable,getUniformBandPowerTable
from project.dataprocess.FolderTree import getDataPath,getOutPath
bandPowerTable=getBandPowerTable(datasetDict['kss_file'],datasetDict['psg_folder'],N_FFT=15*512,HOP_LEN=15*512)
uniformBandaPower=getUniformBandPowerTable(bandPowerTable)


Extracting EDF parameters from D:\dataset\driver_dataset\DROZY\DROZY\psg\1-1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\dataset\driver_dataset\DROZY\DROZY\psg\1-2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\dataset\driver_dataset\DROZY\DROZY\psg\1-3.edf...
EDF file detected
Setting channel info structure...


D:\project_meta\NNproject\NNI\project\dataprocess\SignalProcess.py:199: UserWarning: torch.range is deprecated and will be removed in a future release because its behavior is inconsistent with Python's range builtin. Instead, use torch.arange, which produces values in [start, end).
  startIndex=torch.range(0,((len(y)-N_FFT)//HOP_LENGTH))*HOP_LENGTH


Creating raw.info structure...
Extracting EDF parameters from D:\dataset\driver_dataset\DROZY\DROZY\psg\10-1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\dataset\driver_dataset\DROZY\DROZY\psg\10-3.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\dataset\driver_dataset\DROZY\DROZY\psg\11-1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\dataset\driver_dataset\DROZY\DROZY\psg\11-2.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\dataset\driver_dataset\DROZY\DROZY\psg\11-3.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from D:\dataset\driver_dataset\DROZY\DROZY\psg\12-1.edf...
EDF file detected
Setting channel info struc

In [4]:
id = sorted(uniformBandaPower['ID'].unique())
print(id)
from sklearn.model_selection import KFold
kfold=KFold(n_splits=7)
KFOLD0=list(kfold.split(id))[0]
print(KFOLD0[0])
trainControlTable=uniformBandaPower[uniformBandaPower['ID'].isin(KFOLD0[0])]
testControlTable=uniformBandaPower[uniformBandaPower['ID'].isin(KFOLD0[1])]

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0]
[ 2  3  4  5  6  7  8  9 10 11 12 13]


In [5]:
from project.dataset.BaseDataset import TableDataset
from torch.utils.data import DataLoader
trainDataset=TableDataset(trainControlTable,label='LEVEL',drop=['KSS','ID'])
trainDataloader=DataLoader(trainDataset,batch_size=32)
testDataset=TableDataset(testControlTable,label='LEVEL',drop=['KSS','ID'])
testDataloader=DataLoader(testDataset,batch_size=32)

In [6]:
numFeature=trainDataset[0][0].shape[0]

# 模型


In [18]:
from typing import Any, Optional
import torch
import pytorch_lightning as pl
from pytorch_lightning.utilities.types import STEP_OUTPUT, TRAIN_DATALOADERS
from project.model.CNN1D import simple_cnn1d
from torchmetrics.functional import accuracy
from torch.utils.tensorboard import SummaryWriter

class TrainModel(pl.LightningModule):
    
    def __init__(self, net,lossFunction,optimizer,*args, **kwargs):
        super().__init__()
        self.Writer=SummaryWriter(log_dir=r"D:\project_meta\NNproject\NNI\output\tensorboar_log")
        self.Model=net
        self.LossFunction=lossFunction
        self.Optimizer=optimizer

    def forward(self,x, *args: Any, **kwargs: Any) -> Any:
        y=self.Model(x)
        return y
    def training_step(self, batch,batch_idx,*args: Any, **kwargs: Any) -> STEP_OUTPUT:
        self.Optimizer.zero_grad()
        x,y=batch
        y=y.long()-1
        
        y_hat=self.Model(x.to(torch.float32))
        loss=self.LossFunction(y_hat,y)
        # print(type(x),type(y),type(y_hat))
        # print(x.shape,y.shape,y_hat.shape)
        acc=accuracy(y_hat,y,task='multiclass',num_classes=3)
        pbar={'train_acc':acc}
        # self.Writer.add_scalar('acc',acc,)
        self.log("train_acc", acc, on_step=False, on_epoch=True)
        # self.
        # print(acc)
        self.log("lr", self.Optimizer.state_dict()['param_groups'][0]['lr'], on_step=True, on_epoch=True)
        return {'loss':loss,'progress_bar':pbar}
    def configure_optimizers(self) :
        self.lr_scheduler=torch.optim.lr_scheduler.MultiStepLR(self.Optimizer,milestones=[10,20,30,200],gamma=0.5)
        return [self.Optimizer],[self.lr_scheduler]
    def validation_step(self,batch,batch_idx, *args: Any, **kwargs: Any) -> STEP_OUTPUT | None:
        x,y=batch
        y=y.long()-1
        x=x.to(torch.float32)
        y_hat=self.Model(x)
        loss=self.LossFunction(y_hat,y)
        acc=accuracy(y_hat,y,task='multiclass',num_classes=3)
        self.log("test_acc", acc, on_step=False, on_epoch=True)
        return {'loss':loss}

        

# 训练

In [19]:
from pytorch_lightning import loggers
from torchsummary import summary
from project.model.MLP import MLP

model=MLP(numFeature,[256,64],classes=3)
logger=loggers.TensorBoardLogger(save_dir=r'D:\project_meta\NNproject\NNI\notebook\TraineSummery\EEG\EndToEnd\TESST_LOG_PATH',name='EEG_Feature_MLPIDSplit_LR_SCH')
net=TrainModel(model,torch.nn.CrossEntropyLoss(),optimizer=torch.optim.Adam(model.parameters(),lr=0.0005))
summary(net,(1,30),device='cpu')



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1               [-1, 1, 256]           7,936
              ReLU-2               [-1, 1, 256]               0
            Linear-3                [-1, 1, 64]          16,448
              ReLU-4                [-1, 1, 64]               0
            Linear-5                 [-1, 1, 3]             195
Total params: 24,579
Trainable params: 24,579
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.09
Estimated Total Size (MB): 0.10
----------------------------------------------------------------


In [20]:
trainer=pl.Trainer(logger=logger)
trainer.fit(net,train_dataloaders=trainDataloader,val_dataloaders=testDataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\loops\utilities.py:70: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | Model        | Sequential       | 24.6 K
1 | LossFunction | CrossEntropyLoss | 0     
--------------------------------------------------
24.6 K    Trainable params
0         Non-trainable params
24.6 K    Total params
0.098     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:432: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 20 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\loops\fit_loop.py:280: PossibleUserWarning: The number of training batches (38) is smaller than the logging interval Trainer(log_every_n_steps=

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

g:\install\anaconda\envs\nni\lib\site-packages\pytorch_lightning\trainer\call.py:52: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
